### This notebook contains code for dataset creation.

### Contributors: Developed by Gökalp Özer, Mete Mert Birdal and Abdullah Palaz

In [ ]:
import glob
import time
import os
from os import path
from os import listdir
from os.path import isfile, join
import librosa
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import keras
from keras import models
from keras import layers

import IPython.display as ipd

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
header = 'song_name chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += " polarity"
header = header.split()
header

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

In [ ]:
def polarity(lyric):
    if(sid.polarity_scores(lyric)['pos'] - sid.polarity_scores(lyric)['neg'] > 0.05 ):
        return(1)
    else:
        return(0)

In [ ]:
for fldr in os.listdir('dataset'):
    print(fldr)

In [ ]:
sayac = 1
for folder_name in os.listdir('dataset')[12:]:
    onlyfiles = [f for f in listdir("dataset/" + folder_name) if isfile(join("dataset/" + folder_name, f))]
    onlyfiles.sort(key=lambda v: v.upper())
    matching = [s for s in onlyfiles if "lyrics.csv" in s]
    onlyfiles.remove(matching[0])
    lyrics = pd.read_csv("dataset/" + folder_name + "/" + matching[0], index_col=False, header=None)
    soz_sayaci = 0
    for song_name in onlyfiles:
        y, sr = librosa.load(glob.glob('dataset/' + folder_name + "/" + song_name)[0], duration=90)  
    
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
        if(type(lyrics[0].values[soz_sayaci]) == float):
            sentiment = 0
        else:
            sentiment = polarity(lyrics[0].values[soz_sayaci])
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        file1 = open('data.csv', 'a', newline='')
        with file1:
            writer = csv.writer(file1)
            writer.writerow([song_name] + to_append.split() + [sentiment])


        print("BİTTİ", sayac)
        sayac += 1
        soz_sayaci +=1

    
file1.close()